<a href="https://colab.research.google.com/github/hammy229/SMU-Low-SES-Extraction-Work/blob/main/Arham_Low_ses_simple_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

KeyboardInterrupt: 

**Move to your shared file**

In [ ]:
%cd /content/drive/My Drive/SMU Work

[Errno 2] No such file or directory: '/content/drive/My Drive/SMU Work'
/content


In [ ]:
import os
path = "GroundTruthFilter"
files = [file for file in os.listdir(path) if not file.startswith('.')]

In [ ]:
ground_truth_texts = []

In [ ]:
for file_name in files:
    with open(os.path.join(path, file_name), 'r') as f:
        output = f.read()
        ground_truth_texts.append(output)

In [ ]:
import nltk

nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    tokens = word_tokenize(text.lower())  # Tokenization and lowercasing
    tokens = [word for word in tokens if word.isalpha()]  # Remove non-alphabetic tokens
    tokens = [word for word in tokens if word not in stop_words]  # Remove stopwords
    return ' '.join(tokens)

In [ ]:
ground_truth_texts= [preprocess_text(item) for item in ground_truth_texts]

In [ ]:
my_file = open("not-low-ses.txt", "r")
data = my_file.readlines()
my_file.close()

In [ ]:
len(data)

In [ ]:
not_low_ses = []
for line in data:
    if len(line)>100:
        not_low_ses.append(line)
    if len(not_low_ses) == len(ground_truth_texts):
        break

In [ ]:
len(not_low_ses)

In [ ]:
not_low_ses_texts= [preprocess_text(item) for item in not_low_ses]

In [ ]:
import torch
import torch.nn as nn
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
import numpy as np

In [ ]:
all_text = ground_truth_texts + not_low_ses_texts

In [ ]:
len(all_text)

In [ ]:
all_label = [1 for i in range(len(ground_truth_texts))] + [-1 for i in range(len(not_low_ses_texts))]

In [ ]:
len(all_label)

In [ ]:
# all_text

# **Simple Classifier**

In [ ]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()

all_text = np.array(all_text)
all_label = np.array(all_label)

train_texts, val_texts, train_labels, val_labels = train_test_split(all_text, all_label, test_size=0.2, random_state=42, shuffle=True)

In [ ]:
vectorizer.fit(train_texts)
X_train = vectorizer.transform(train_texts)
X_test  = vectorizer.transform(val_texts)

In [ ]:
from sklearn.linear_model import LogisticRegression

classifier = LogisticRegression()
classifier.fit(X_train, train_labels)
score = classifier.score(X_test, val_labels)

print("Accuracy:", score)


In [ ]:
classifier.predict_proba(X_train[0:3])

In [ ]:
text = "it's a nice place. if you don't have friends it can be hard to find fun things to do on the weekends, so definitely make an effort early on to meet interesting people so that you aren't stuck alone in your dorm on the weekends. it pretty easy to navigate, but there is an nku app you should definitely download that you can use for a lot of things like looking at grades and bus schedules. if you don't have a car learn the bus lines. leave campus often to explore cincinnati. campus can be quiet sometimes, but you will never run out of fun things to do in the city. join clubs specific to your major to meet people. intramurals are a pretty big part of greek life here as well. walk around buildings where you don't have classes, you never know what you might stumble upon. the dry campus policy is pretty strict, so if you want to drink try to find some friends with apartments off campus. people tend to be nice and helpful. if you have a meal plan, einsteins in the library is the best place to get lunch, but they only accept swipes after 1 pm. the norse tech bar is awesome, utilize it. they can fix your computer, and you can even check out equipment such as cameras and ipads. that's all i can think of for now, hope you enjoy nku."


In [ ]:
def LRPredict():
    predictions = []
    for i, text in enumerate(ground_truth_texts):
        text = preprocess_text(text)
        vec_text = vectorizer.transform([text])
        prob = classifier.predict_proba(vec_text)
        if prob[0][1]>0.90:
            predictions.append(f"Text {i+1}: low_ses")
        else:
            predictions.append(f"Text {i+1}: not_low_ses")
    return predictions

predictions = LRPredict()
for prediction in predictions:
    print(prediction)

Incorrect classification for Now_Low_Ses - 5, 12, etc

In [ ]:
def LRPredict(text):
    text = preprocess_text(text)
    vec_text = vectorizer.transform([text])
    prob = classifier.predict_proba(vec_text)
    if prob[0][1]>0.90:
        return "low_ses"
    else:
        return "not_low_ses"

In [ ]:
LRPredict(text)

In [ ]:
import os

def classify_ses(text):
    low_ses_keywords = [
        "poor", "low income", "unemployed", "financial aid", "pell grant",
        "food pantry", "working multiple jobs", "low wage", "scholarships",
        "struggling to pay", "couldn't afford to", "no money for", "in debt", "relying on grants",
        "parents can't afford to", "financial hardship", "living in poverty"
    ]

    text_lower = text.lower()

    for keyword in low_ses_keywords:
        if keyword in text_lower:
            return "Low SES"

    return "Not Low SES"

def process_files(directory):
    results = {}

    for filename in os.listdir(directory):
        if filename.endswith(".txt"):
            filepath = os.path.join(directory, filename)
            with open(filepath, 'r', encoding='utf-8') as file:
                content = file.read()
                # Split the file content by newlines and remove any empty entries
                entries = [entry for entry in content.split('\n') if entry.strip()]  # Filter out empty entries
                entry_results = []
                for i, entry in enumerate(entries):
                    classification = classify_ses(entry)
                    entry_results.append((i+1, classification, entry[:100]))  # Store entry index, classification, and snippet
                    print(f"Processing entry {i+1} in file {filename}")
                    print(f"Content snippet: {entry[:100]}...")  # Print first 100 characters for context
                    print(f"Classification: {classification}")
                results[filename] = entry_results

    return results


def main():
    directory = "/content/drive/My Drive/SMU Work"
    results = process_files(directory)

    for filename, entries in results.items():
        print(f"\nResults for {filename}:")
        for index, classification, snippet in entries:
            print(f"Entry {index}: {classification} - Snippet: {snippet}...")
        print("\n")

if __name__ == "__main__":
    main()


In [ ]:
 from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/My Drive/SMU Work

import os
path = "GroundTruthFilter"
files = [file for file in os.listdir(path) if not file.startswith('.')]

ground_truth_texts = []

for file_name in files:
    with open(os.path.join(path, file_name), 'r') as f:
        output = f.read()
        ground_truth_texts.append(output)

import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    tokens = word_tokenize(text.lower())  # Tokenization and lowercasing
    tokens = [word for word in tokens if word.isalpha()]  # Remove non-alphabetic tokens
    tokens = [word for word in tokens if word not in stop_words]  # Remove stopwords
    return ' '.join(tokens)

ground_truth_texts= [preprocess_text(item) for item in ground_truth_texts]

my_file = open("not-low-ses.txt", "r")
data = my_file.readlines()
my_file.close()

not_low_ses = []
for line in data:
    if len(line)>100:
        not_low_ses.append(line)
    if len(not_low_ses) == len(ground_truth_texts):
        break

not_low_ses_texts= [preprocess_text(item) for item in not_low_ses]

import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

all_text = ground_truth_texts + not_low_ses_texts
all_label = [1 for i in range(len(ground_truth_texts))] + [-1 for i in range(len(not_low_ses_texts))]

train_texts, val_texts, train_labels, val_labels = train_test_split(all_text, all_label, test_size=0.2, random_state=42, shuffle=True)

vectorizer = CountVectorizer()
vectorizer.fit(train_texts)
X_train = vectorizer.transform(train_texts)
X_test  = vectorizer.transform(val_texts)

rfc = RandomForestClassifier(n_estimators=100, random_state=42)
rfc.fit(X_train, train_labels)

score = rfc.score(X_test, val_labels)
print("Accuracy:", score)

def LRPredict(text):
    text = preprocess_text(text)
    vec_text = vectorizer.transform([text])
    prob = rfc.predict(vec_text)
    if prob[0]==1:
        return "low_ses"
    else:
        return "not_low_ses"

text = "it's a nice place. if you don't have friends it can be hard to find fun things to do on the weekends, so definitely make an effort early on to meet interesting people so that you aren't stuck alone in your dorm on the weekends. it pretty easy to navigate, but there is an nku app you should definitely download that you can use for a lot of things like looking at grades and bus schedules. if you don't have a car learn the bus lines. leave campus often to explore cincinnati. campus can be quiet sometimes, but you will never run out of fun things to do in the city. join clubs specific to your major to meet people. intramurals are a pretty big part of greek life here as well. walk around buildings where you don't have classes, you never know what you might stumble upon. the dry campus policy is pretty strict, so if you want to drink try to find some friends with apartments off campus. people tend to be nice and helpful. if you have a meal plan, einsteins in the library is the best place to get lunch, but they only accept swipes after 1 pm. the norse tech bar is awesome, utilize it. they can fix your computer, and you can even check out equipment such as cameras and ipads. that's all i can think of for now, hope you enjoy nku."

print(LRPredict(text))

def classify_ses(text):
    low_ses_keywords = [
        "poor", "low income", "unemployed", "financial aid", "pell grant",
        "food pantry", "working multiple jobs", "low wage", "scholarships",
        "struggling to pay", "couldn't afford to", "no money for", "in debt", "relying on grants",
        "parents can't afford to", "financial hardship", "living in poverty"
    ]

    text_lower = text.lower()

    for keyword in low_ses_keywords:
        if keyword in text_lower:
            return "Low SES"

    return "Not Low SES"

def process_files(directory):
    results = {}

    for filename in os.listdir(directory):
        if filename.endswith(".txt"):
            filepath = os.path

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/SMU Work


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Accuracy: 0.6538461538461539
not_low_ses


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/My Drive/SMU Work

import os
path = "GroundTruthFilter"
files = [file for file in os.listdir(path) if not file.startswith('.')]

ground_truth_texts = []

for file_name in files:
    with open(os.path.join(path, file_name), 'r') as f:
        output = f.read()
        ground_truth_texts.append(output)

import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    tokens = word_tokenize(text.lower())  # Tokenization and lowercasing
    tokens = [word for word in tokens if word.isalpha()]  # Remove non-alphabetic tokens
    tokens = [word for word in tokens if word not in stop_words]  # Remove stopwords
    return ' '.join(tokens)

ground_truth_texts= [preprocess_text(item) for item in ground_truth_texts]

my_file = open("not-low-ses.txt", "r")
data = my_file.readlines()
my_file.close()

not_low_ses = []
for line in data:
    if len(line)>100:
        not_low_ses.append(line)
    if len(not_low_ses) == len(ground_truth_texts):
        break

not_low_ses_texts= [preprocess_text(item) for item in not_low_ses]

import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import svm
from sklearn.model_selection import train_test_split

all_text = ground_truth_texts + not_low_ses_texts
all_label = [1 for i in range(len(ground_truth_texts))] + [-1 for i in range(len(not_low_ses_texts))]

train_texts, val_texts, train_labels, val_labels = train_test_split(all_text, all_label, test_size=0.2, random_state=42, shuffle=True)

vectorizer = CountVectorizer()
vectorizer.fit(train_texts)
X_train = vectorizer.transform(train_texts)
X_test  = vectorizer.transform(val_texts)

svm_model = svm.SVC(kernel='linear', C=1)
svm_model.fit(X_train, train_labels)

score = svm_model.score(X_test, val_labels)
print("Accuracy:", score)

def LRPredict(text):
    text = preprocess_text(text)
    vec_text = vectorizer.transform([text])
    prob = svm_model.predict(vec_text)
    if prob[0]==1:
        return "low_ses"
    else:
        return "not_low_ses"

text = "it's a nice place. if you don't have friends it can be hard to find fun things to do on the weekends, so definitely make an effort early on to meet interesting people so that you aren't stuck alone in your dorm on the weekends. it pretty easy to navigate, but there is an nku app you should definitely download that you can use for a lot of things like looking at grades and bus schedules. if you don't have a car learn the bus lines. leave campus often to explore cincinnati. campus can be quiet sometimes, but you will never run out of fun things to do in the city. join clubs specific to your major to meet people. intramurals are a pretty big part of greek life here as well. walk around buildings where you don't have classes, you never know what you might stumble upon. the dry campus policy is pretty strict, so if you want to drink try to find some friends with apartments off campus. people tend to be nice and helpful. if you have a meal plan, einsteins in the library is the best place to get lunch, but they only accept swipes after 1 pm. the norse tech bar is awesome, utilize it. they can fix your computer, and you can even check out equipment such as cameras and ipads. that's all i can think of for now, hope you enjoy nku."

print(LRPredict(text))

def classify_ses(text):
    low_ses_keywords = [
        "poor", "low income", "unemployed", "financial aid", "pell grant",
        "food pantry", "working multiple jobs", "low wage", "scholarships",
        "struggling to pay", "couldn't afford to", "no money for", "in debt", "relying on grants",
        "parents can't afford to", "financial hardship", "living in poverty"
    ]

    text_lower = text.lower()

    for keyword in low_ses_keywords:
        if keyword in text_lower:
            return "Low SES"

    return "Not Low SES"

def process_files(directory):
    results = {}
    for filename in os.listdir(directory):
        if filename.endswith('.txt'):
            # Add your logic here to process the file
            pass # This is a placeholder, replace it with your file processing code

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/SMU Work


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Accuracy: 0.8076923076923077
not_low_ses
